# Big project: Patient-nurse-room assignment

In this project, you are required to solve a healthcare scheduling problem involving the assignment of patients to rooms and nurses to rooms in a hospital. The goal is to develop an optimization model using mip that minimizes the total delay in patient admissions while adhering to a set of operational and resource constraints. This problem is inspired by the Integrated Healthcare Timetabling Competition 2024 (https://ihtc2024.github.io/).

The hospital operates with a set of patients, rooms, and nurses, each with specific characteristics. Patients are defined by their release date, indicating the earliest day they can be admitted, and a due date, specifying the latest allowable admission day. Patients also have a required length of stay. Rooms are characterized by fixed capacities that determine the maximum number of patients they can accommodate at any given time. Additionally, some patients may have incompatibilities with certain rooms, restricting their assignment.

Nurses have predefined rosters detailing their availability across days. Each occupied room must have one assigned nurse on every day of the scheduling horizon. Nurses can be assigned to a maximum of 3 rooms in a given day.

The problem requires assigning each patient to a room and ensuring that every occupied room has an assigned nurse during each day. These assignments must satisfy several constraints: no room may exceed its capacity, nurse assignments must respect their availability, and patients must only be assigned to compatible rooms. The primary objective is to minimize the total delay in admissions, defined as the sum of the differences between actual admission dates and the release dates of all patients. In what follows we detail the constraints and decisions.


**Constraints:**

- A patient’s length of stay must be respected, meaning they occupy a room for the full duration of their stay.
- A room cannot exceed its capacity on any given day.
- Patients can only be assigned to rooms that they are compatible with.
- Each occupied room must have exactly one assigned nurse per day.
- Nurses can only be assigned to rooms on days when they are available.
- Nurses can be assigned to a maximum of 3 different rooms in the same day.
- Admission days must be within the specified release and due dates.

**Decisions:**

- Assign each patient to a room, ensuring that the same room is assigned for the duration of their stay.
- Determine the admission day for each patient within the allowed range of their release and due dates.
- Assign one nurse to each occupied room on every day of the scheduling period.

**Objective:**

The objective is to minimize the total admission delay, defined as the sum of the differences between the admission day and the release date of patients.


**Sumbmission instructions:**

- Each group can be formed by maximum 3 students
- One sumbission per group
- Submission consists of uploading a .ipynb file with the following name format: "LastName1_LastName2_LastName3.ipynb" where the name of the file contains the last names of the students in alphabetic order.
- The submitted file must contain:
  - Name of students
  - Python code using mip library with the model and print out of the objective value in one single code cell
- Questions regarding the project description will be answered in the Forum "Big project" (Please feel free to email me in case the answer is not addressed in the forum at maximiliano.cubillos@polimi.it, after the holiday period)
- Submission deadline:  February 10th 00:00 hrs. No late admissions will be considered.

**Student names**


- ACQUADRO PATRIZIO (11087897)
- DRUGMAN TITO NICOLA (10847631)


In [ ]:
# --------------------------------------------- IMPORTING ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ IMPORTING /\/\/\/\/\/\/\/\/")
print("")


import importlib
import cffi
importlib.reload(cffi)
!pip install mip

import requests
import pandas as pd
from mip import Model, BINARY, xsum, minimize, OptimizationStatus
from IPython.display import display

# --------------------------------------------- LOAD DATA ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ LOAD DATA /\/\/\/\/\/\/\/\/")
print("")


# load json
data_url = "https://raw.githubusercontent.com/TitoNicolaDrugman/FOR-Patient-nurse-room-assignment/refs/heads/main/test02.json"
response = requests.get(data_url)
if response.status_code != 200:
    raise ValueError("Failed to download JSON data from the provided URL")
data = response.json()

# DAYS
num_days = data.get("days", None)
if num_days is None:
    raise KeyError("The 'days' key is missing from the JSON data.")
days = range(num_days)

# ROOMS
rooms_data = data.get("rooms", [])
if not rooms_data:
    raise KeyError("No 'rooms' data found in the JSON file.")
rooms = [room['id'] for room in rooms_data]
room_capacity = {room['id']: room['capacity'] for room in rooms_data}

# PATIENT
patients_data = data.get("patients", [])
if not patients_data:
    raise KeyError("No 'patients' data found in the JSON file.")
patients = [p['id'] for p in patients_data]
patient_release = {p['id']: p['release_date'] for p in patients_data}
patient_due = {p['id']: p['due_date'] for p in patients_data}
patient_length_of_stay = {p['id']: p['length_of_stay'] for p in patients_data}
patient_incompatible_rooms = {p['id']: p['incompatible_room_ids'] for p in patients_data}

# NURSE
nurses_data = data.get("nurses", [])
if not nurses_data:
    raise KeyError("No 'nurses' data found in the JSON file.")
nurses = [n['id'] for n in nurses_data]
nurse_shifts = {n['id']: [shift['day'] for shift in n['working_shifts']] for n in nurses_data}

rooms_df = pd.DataFrame(rooms_data)
patients_df = pd.DataFrame(patients_data)
nurses_df = pd.DataFrame(nurses_data)


# sanity check
print(rooms_df.head())
print("-"*20)
print(patients_df.head())
print("-"*20)
print(nurses_df.head())
print("-"*20)


# --------------------------------------------- DECISION VARIABLES ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ DECISION VARIABLES /\/\/\/\/\/\/\/\/")
print("")


"""**x(i,r,d)** dummy decision variable
- i: patient, r: room, d: day
- for patient admission
- is 1 if patient i have admission on room r on day d, 0 otherwise
- the same room must be assigend for the entire duration

**y(r,d,n)** dummy decision variable
- r: room, d: day, n: nurse
- for room-nurse assigment
- is 1 if nurse n is assigend on room r on day d
"""


model = Model("Healthcare_Scheduling_BigProject", solver_name="CBC")

x = {
    (i, r, d): model.add_var(var_type=BINARY, name=f"x_{i}_{r}_{d}")
    for i in patients
    for r in rooms
    for d in days
}

y = {
    (r, d, n): model.add_var(var_type=BINARY, name=f"y_{r}_{d}_{n}")
    for r in rooms
    for d in days
    for n in nurses
}

print(len(x)) # it should be 700 since there are 20 patients, 5 rooms and 7 days => 20*5*7 = 700
print(len(y)) # it should be 175 since there are 5 rooms, 7 days and 5 nurses => 5*7*5 = 175


# --------------------------------------------- CONSTRAINTS ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ CONSTRAINTS /\/\/\/\/\/\/\/\/")
print("")


# ensure that each patient is admitted exaclty one
# for each patient i the sum of all x[i,r,d] (changing room r and day d) must be 1
for i in patients:
    model += xsum(x[i, r, d]
                  for r in rooms
                  for d in days) == 1, f"One_Admission_{i}"

#----------------------------------------------------------------------------------------------

# ensure that the capacity of each room each day is satisfied
# for each room, for each day the total number of patients in the room can not exceed the room capacity
# first we need to consider that if x[i,r,d] = 1 then the patient occupy that room that day but also all the other days for the length_of_stay
# a patient i admitted on day [d] will stay in the room for patient_length_of_stay[i] days
for r in rooms:
    for t in days:
        model += xsum(
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t and t < d + patient_length_of_stay[i]
        ) <= room_capacity[r], f"Capacity_{r}_Day_{t}"

#----------------------------------------------------------------------------------------------

# ensure that if a patient i can not stay in room r then for all days x[i,r,d] must be 0
for i in patients:
    for r in rooms:
        if r in patient_incompatible_rooms[i]:
            for d in days:
                model += x[i, r, d] == 0, f"Incompatible_{i}_{r}_{d}"

#----------------------------------------------------------------------------------------------

# ensure that forall patient i the admission is between release day and due date
# make 0 everything before patient_release and patient_due
for i in patients:
    for r in rooms:
        for d in days:
            if d < patient_release[i] or d > patient_due[i]:
                model += x[i, r, d] == 0, f"Outside_Window_{i}_{r}_{d}"

#----------------------------------------------------------------------------------------------

# ensure that each nurse for each room can work only during his/her shift
# a nurse can only work on day d if d is in nurse_shifts[n]
# so is 0 if d is NOT in nurse_shifts[n]
for n in nurses:
    for r in rooms:
        for d in days:
            if d not in nurse_shifts[n]:
                model += y[r, d, n] == 0, f"Nurse_Not_Available_{n}_{r}_{d}"

#----------------------------------------------------------------------------------------------

# ensure that each nurse can be assigned to at most 3 rooms per day
for n in nurses:
    for d in days:
        model += xsum(y[r, d, n] for r in rooms) <= 3, f"Nurse_Max3Rooms_{n}_{d}"

#----------------------------------------------------------------------------------------------

# Auxiliary binary variable: 1 if room r is occupied on day t, 0 otherwise
occupied = {
    (r, t): model.add_var(var_type=BINARY, name=f"occupied_{r}_{t}")
    for r in rooms
    for t in days
}

for r in rooms:
    for t in days:
        occupancy_expr = xsum(  # how many patients are occupying room r on day t
            x[i, r, d]
            for i in patients
            for d in days
            if d <= t and t < d + patient_length_of_stay[i]
        )
        # if occupied[r,t] = 1 then there must be at least one patient occupying the room on that day (occupancy_expr >= 1)
        # if occupied[r,t] = 0 then (occupancy_expr >= 0) nothing imposed
        model += occupancy_expr >= occupied[r, t], f"Occupancy_lower_{r}_{t}"

        # how many patients can occupy room r in day t
        # occupancy_expr (how many patients occupy room r on day t)
        # if occupied[r, t] = 0 (no patient that day on that room) then we have occupancy_expr <= 0
        # if occupied[r, t] = 1 (at least one patient that day on that room) then we have occupancy_expr <= 1
        model += occupancy_expr <= room_capacity[r] * occupied[r, t], f"Occupancy_upper_{r}_{t}"

#----------------------------------------------------------------------------------------------

# ensure that in each room each day the number of nurses assigned to that room is 1 or 0
for r in rooms:
    for t in days:
        model += xsum(y[r, t, n]
                      for n in nurses) == occupied[r, t], f"Nurse_Assignment_{r}_{t}"



# --------------------------------------------- OBJECTIVE FUNCTION ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ OBJECTIVE FUNCTION /\/\/\/\/\/\/\/\/")
print("")


"""
**Task:** minimize the total delay in admissions, defined as the sum of the differences between actual admission dates and the release dates of all patients.

=> minimize SUM[(admission_day(i) - release_date(i))] for all patients i

To compute the admission delay we do *(d - patient_release[i]) * x[i, r, d]* since *x[i, r, d] = 1* if patient i is admitted on room r on day d, otherwise is 0

Example:
Suppose that we have a patient (*id 100*) which have release date 0 and due date 3. The patient could be admitted:
- Day 0: thus admitted - release = 0 - 0 <= no delay
- Day 1: thus admitted - release = 1 - 0 <= 1 delay
- ...
"""

admission_delay = xsum(
    (d - patient_release[i]) * x[i, r, d]
    for i in patients
    for r in rooms
    for d in days
    if d >= patient_release[i] and d <= patient_due[i]
)
model.objective = minimize(admission_delay)


# --------------------------------------------- CHECKING ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ CHECKING /\/\/\/\/\/\/\/\/")
print("")

status = model.optimize()

if status == OptimizationStatus.OPTIMAL:
    print("Optimal solution found")
    print(f"Total admission delay = {model.objective_value}")

    for i in patients:
        for r in rooms:
            for d in days:
                if x[i, r, d].x == 1:
                    print(f"Patient {i} assigned to Room {r} with admission Day {d}. (Stay length = {patient_length_of_stay[i]})")

    print("\nNurse assignments:")
    for r in rooms:
        for d in days:
            for n in nurses:
                if y[r, d, n].x == 1:
                    print(f"Nurse {n} assigned to Room {r} on Day {d}")

elif status == OptimizationStatus.FEASIBLE:
    print("A feasible solution was found (not guaranteed optimal)")
    print(f"Objective value (admission delay) = {model.objective_value}")
else:
    print("No feasible solution found")



# --------------------------------------------- TABLE RESULT ---------------------------------------------
print("")
print("/\/\/\/\/\/\/\/\ TABLE RESULT /\/\/\/\/\/\/\/\/")
print("")

table_data = []
for t in days:
    row_data = []
    for r in rooms:
        patients_in_room = []
        for i in patients:
            for d in days:
                if x[i, r, d].x ==1 and t in range(d, d + patient_length_of_stay[i]):
                    patients_in_room.append(i)
                    break

        nurse_assigned = None
        for n in nurses:
            if y[r, t, n].x > 0.5:
                nurse_assigned = n
                break

        if patients_in_room:
            cell_content = f"Patient(s): {', '.join(map(str, patients_in_room))} Nurse: {nurse_assigned}"
        else:
            cell_content = "Empty"
        row_data.append(cell_content)
    table_data.append(row_data)

schedule_df = pd.DataFrame(
    table_data,
    index=[f"Day {t}" for t in days],
    columns=[f"Room {r}" for r in rooms]
)

display(schedule_df)




/\/\/\/\/\/\/\/\ IMPORTING /\/\/\/\/\/\/\/\/


/\/\/\/\/\/\/\/\ LOAD DATA /\/\/\/\/\/\/\/\/

   id  capacity
0   0         3
1   1         2
2   2         2
3   3         2
4   4         2
--------------------
   id  length_of_stay  release_date  due_date incompatible_room_ids
0   0               3             0         3                    []
1   1               2             0         4                   [1]
2   2               4             0         5                   [2]
3   3               3             0         4                   [0]
4   4               2             1         6                    []
--------------------
   id                                    working_shifts
0   0  [{'day': 0}, {'day': 1}, {'day': 2}, {'day': 3}]
1   1  [{'day': 1}, {'day': 2}, {'day': 3}, {'day': 4}]
2   2  [{'day': 0}, {'day': 1}, {'day': 3}, {'day': 4}]
3   3  [{'day': 2}, {'day': 3}, {'day': 5}, {'day': 6}]
4   4              [{'day': 4}, {'day': 5}, {'day': 6}]
--------------------

/\

,Room 0,Room 1,Room 2,Room 3,Room 4
Day 0,"Patient(s): 16, 19 Nurse: 2",Patient(s): 6 Nurse: 0,"Patient(s): 1, 3 Nurse: 2","Patient(s): 0, 8 Nurse: 0",Patient(s): 2 Nurse: 0
Day 1,"Patient(s): 16, 19 Nurse: 0","Patient(s): 4, 13 Nurse: 1","Patient(s): 1, 3 Nurse: 0","Patient(s): 0, 8 Nurse: 1","Patient(s): 2, 5 Nurse: 0"
Day 2,"Patient(s): 7, 14, 19 Nurse: 1","Patient(s): 4, 13 Nurse: 0","Patient(s): 3, 9 Nurse: 0","Patient(s): 0, 8 Nurse: 1","Patient(s): 2, 5 Nurse: 0"
Day 3,"Patient(s): 7, 14 Nurse: 2",Patient(s): 15 Nurse: 0,Patient(s): 9 Nurse: 0,Patient(s): 10 Nurse: 0,"Patient(s): 2, 5 Nurse: 1"
Day 4,"Patient(s): 11, 12, 14 Nurse: 1",Patient(s): 17 Nurse: 1,Patient(s): 9 Nurse: 2,Patient(s): 10 Nurse: 4,Empty
Day 5,"Patient(s): 11, 18 Nurse: 3",Patient(s): 17 Nurse: 3,Empty,Empty,Empty
Day 6,Patient(s): 18 Nurse: 3,Patient(s): 17 Nurse: 3,Empty,Empty,Empty
